# ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [54]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [58]:
# checking current working directory
print(os.getcwd())

# Getting the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'    

print(filepath)

/home/chinanu/UDACITY/Data Engineering (udacity)/etl_pipeline_for_music_streaming_app
/home/chinanu/UDACITY/Data Engineering (udacity)/etl_pipeline_for_music_streaming_app/event_data


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [56]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 

# Creating a for loop in order to generate a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):

# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    
# for every filepath in the file path list 
    for f in file_path_list:

# reading csv file 
        with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
            # creating a csv reader object 
            csvreader = csv.reader(csvfile) 
            next(csvreader)
        
 # extracting each data row and append it        
            for line in csvreader:                
                full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [57]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
<br>

### Task 1: Getting the artist, song title and song's length where sessionId = 338, and itemInSession = 4

#### i. Creating *music_history_by_sessionId_and_itemInSession* table with Partition Key *sessionId* and *itemInSession* as Primary Key

In [34]:
query = """CREATE TABLE music_history_by_sessionId_and_itemInSession 
            (
                sessionId int, 
                itemInSession int, 
                artist text, 
                song text, 
                length float, 
                PRIMARY KEY((sessionId, itemInSession))
            );"""     

try:
    session.execute(query)
except Exception as e:
    print(e)              

#### ii. Load data from the required columns in *event_datafile_new.csv* file into *music_history_by_sessionId_and_itemInSession* table in Apache Cassandra  

In [35]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_history_by_sessionId_and_itemInSession (sessionId, itemInSession, artist, song, length) "
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### iii.   Retrieve record for artist, song title and song's length where sessionId = 338, and itemInSession = 4 from *music_history_by_sessionId_and_itemInSession*

In [36]:
query = "SELECT artist, song, length FROM music_history_by_sessionId_and_itemInSession where sessionId = 338 and itemInSession=4;"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Task 2: Getting the artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### i. Creating *music_history_by_userid_and_sessionid* table with *userId* and *sessionId* as Partition Key, and *itemInSession* as Clustering Key, which all make up the Primary Key.<br>
##### Partition Key - is the primary look up to find rows. Clustering Key - is for ordering within a partition.

In [37]:
query = """CREATE TABLE music_history_by_userid_and_sessionid 
            (
                userId int, 
                sessionId int, 
                itemInSession int,
                artist text, 
                song text, 
                user text,
                PRIMARY KEY((userId, sessionId), itemInSession)
            );"""     

try:
    session.execute(query)
except Exception as e:
    print(e)                               

#### ii. Load data from the required columns in *event_datafile_new.csv* file into *music_history_by_sessionId_and_itemInSession* table in Apache Cassandra  

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_history_by_userid_and_sessionid (userId, sessionId, itemInSession, artist, song, user) "
        query = query + " VALUES(%s, %s, %s, %s, %s, %s)"
        user = " ".join([line[1], line[4]])
        ## Assign which column element should be assigned for each column in the INSERT statement.  
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], user))

#### iii.   Retrieve record for artist, song (sorted by itemInSession) and user (first and last name) Where userid = 10, sessionid = 182 from *music_history_by_userid_and_sessionid*

In [39]:
query = "SELECT artist, song, user FROM music_history_by_userid_and_sessionid where userId = 10 and sessionId=182 ORDER BY itemInSession;"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Task 3: Getting every user name (first and last name) who listened to the song 'All Hands Against His Own'. 

#### i. Creating *music_history_by_song* table with *song* as Partition Key, and *userid* as Clustering Key, which all make up the Primary Key.

In [45]:
query = """CREATE TABLE music_history_by_song 
            (
                song text,
                userid int, 
                user text,
                PRIMARY KEY(song, userid)
            );"""     

try:
    session.execute(query)
except Exception as e:
    print(e)            

                    

#### ii. Load data from the required columns in *event_datafile_new.csv* file into *music_history_by_song* table in Apache Cassandra

In [48]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_history_by_song (song, user, userid) "
        query = query + " VALUES(%s, %s, %s)"
        user = " ".join([line[1], line[4]])
        session.execute(query, (line[9], user, int(line[10])))

#### iii.   Retrieve records for users (first and last name) Where song = 'All Hands Against His Own' from *music_history_by_song*

In [49]:
query = "SELECT user FROM music_history_by_song WHERE song='All Hands Against His Own';"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.user)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [50]:
try:
    session.execute("DROP TABLE music_history_by_sessionId_and_itemInSession;")
except Exception as e:
    print(e)

In [51]:
try:
    session.execute("DROP TABLE music_history_by_userid_and_sessionid;")
except Exception as e:
    print(e)

In [52]:
try:
    session.execute("DROP TABLE music_history_by_song;")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [53]:
session.shutdown()
cluster.shutdown()